In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
import cv2
import re
import nltk
nltk.download('popular')
###############################
import pytesseract


import matplotlib.pyplot as plt
from PIL import Image
import os

import random

import warnings
warnings.filterwarnings(action='ignore')


import skimage
from skimage.feature import greycomatrix, greycoprops
from skimage.filters import sobel

In [ ]:
#https://www.kaggle.com/cdeotte/part-2-rapids-tfidfvectorizer-cv-0-700

## Data Set

In [ ]:
##https://www.kaggle.com/cdeotte/part-2-rapids-tfidfvectorizer-cv-0-700
train = pd.read_csv('../input/shopee-product-matching/train.csv')
train

In [ ]:
train = pd.read_csv('../input/shopee-product-matching/train.csv')
answer=train[['posting_id','label_group']].groupby('label_group').posting_id.agg('unique').to_dict()
train['answer']=train.label_group.map(answer)
train

In [ ]:
def img_show(row_num):

    image_name=train['image'].loc[row_num]

    img = os.path.join('../input/shopee-product-matching/train_images/', image_name)

    image = cv2.imread(img)

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(5, 5))

    plt.imshow(image)

    plt.title(train['title'].loc[row_num])

In [ ]:
def group_show(row_num):

    lb_group=train['label_group'].loc[row_num]
    
    index_group=train[train['label_group']==lb_group].index
    
    for ele in index_group:
        
        img_show(ele)

## data overview

In [ ]:
train[train['label_group']==train.loc[10]['label_group']]

In [ ]:
group_show(10)

## SHOPEE data analysis 
### pictures and titles by group

In [ ]:
group_show(0)

In [ ]:
group_show(1)

In [ ]:
# Some titles are with upper  case, others are mixed. Need to arrage it.
group_show(3)

In [ ]:
#if we extract the strings from the picture 'Nescafe' it will be helpful.
group_show(4)

In [ ]:
def img_show_other(row_num):

    image_name=train['image'].loc[row_num]

    img = os.path.join('../input/shopee-product-matching/train_images/', image_name)

    image = cv2.imread(img)

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(5, 5))

    plt.imshow(image)

    plt.title(train['title'].loc[row_num])

In [ ]:
import os
def filtered_img(row_num):
    image_name=train['image'].loc[row_num]
    img = os.path.join('../input/shopee-product-matching/train_images/', image_name)
    img
    image = cv2.imread(img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(sobel(image[:,:,2]))
filtered_img(1111)

In [ ]:
max_num=0
for idx,ele in enumerate(train['answer']):
    if max_num<len(ele):
        max_num = len(ele)
        index_ = idx
print('Index number:',index_,' has',max_num,' same items (maximum number)')
# group_show(50)

## SHOPEE data analysis 
### pictures

In [ ]:

row_num=110
image_name=train['image'].loc[row_num]
img = os.path.join('../input/shopee-product-matching/train_images/', image_name)
image = cv2.imread(img)
print(image.shape)
plt.imshow(image[:,:])
print(sobel(image[:,:,2]).shape)


In [ ]:
plt.imshow(sobel(image[:,:,2]))

## SHOPEE data analysis 
### the WordCloud with titles

In [ ]:
from wordcloud import WordCloud, STOPWORDS
stopwords = nltk.corpus.stopwords.words("english")
wordcloud = WordCloud(width = 800, 
                      height = 800,
                      background_color ='white',
                      min_font_size = 10,
                      stopwords = stopwords,).generate(' '.join(train['title'])) 

# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 

plt.show() 

## Feature Processing

### Text 

In [ ]:
#https://www.kaggle.com/ishandutta/v7-shopee-indepth-eda-one-stop-for-all-your-needs
def preprocess_text(text, flg_stemm=False, flg_lemm=True):

    lst_stopwords = nltk.corpus.stopwords.words("english")
    
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', ' ', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()    
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

In [ ]:
train["clean_title"] = train["title"].apply(lambda x: preprocess_text(x, flg_stemm=False, flg_lemm=True ))

In [ ]:
train[["clean_title","title"]]

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
clean_title=train["clean_title"]
title=train['title']
fdist=FreqDist()
fdist_title=FreqDist()
for ele in clean_title:
    ele_tok=word_tokenize(ele)
    for tok_word in ele_tok:
        fdist[tok_word]+=1
for ele in title:
    ele_tok=word_tokenize(ele)
    for tok_word in ele_tok:
        fdist_title[tok_word]+=1
print('top 10 tokens from clean_title:',fdist.most_common(10))

print('top 10 tokens from title:',fdist_title.most_common(10))

In [ ]:
# fdist.most_common(400)

In [ ]:
def abuse_score(keyword): 
    arr=[]
    for idx,ele in enumerate(clean_title):
        ele_tok=word_tokenize(ele)
        for tok_word in ele_tok:
            if tok_word == keyword:
                arr.append(idx)
                break
    return len(arr)/fdist[keyword]

In [ ]:
abuse_score('original')

In [ ]:
abuse_score('serum')

In [ ]:
abuse_score('puzzle')

In [ ]:
# abuse_score calculator
# it takes lots of time so I uploaeded the result of it 


# result = []
# for ele in fdist.most_common(300):
#     print(ele)
#     result.append((ele[0],abuse_score(ele[0])))
# result_pd = pd.DataFrame(result) 

#https://www.kaggle.com/ishandutta/v7-shopee-indepth-eda-one-stop-for-all-your-needs

result_pd=pd.read_csv('../input/shop-result/result.csv')
def preprocess2_text(text,crite):
    lst_stopwords = result_pd[result_pd['1']>crite]['0']

    ## Tokenize (convert from string to list)
    lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
    text = " ".join(lst_text)
    return text


In [ ]:
result_pd[result_pd['1']>0.98]['0']

In [ ]:
#Tesseract Library
train["clean_title_remove_98"] = train["clean_title"].apply(lambda x: preprocess2_text(x,0.94))

In [ ]:
train[["clean_title","title","clean_title_remove_98"]]

## TEXT EXTRACT FROM PICTURES

In [ ]:
image_id = random.choice(train['image'].values)
print(image_id)
img_path = os.path.join('../input/shopee-product-matching/train_images/', image_id)
image = cv2.imread(img_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(5, 5))

plt.imshow(image)
plt.title(image_id)
print(' OCR result : ' + pytesseract.image_to_string(Image.open(img_path), timeout=10))
print('======================')
print(' GT title column is : ' + train[train['image'] == image_id]['title'].values)

In [ ]:
import easyocr
def img2txt(image_id):
    img_path = os.path.join('../input/shopee-product-matching/train_images/', image_id)
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = pytesseract.image_to_string(Image.open(img_path), timeout=10)
    #print(' OCR result : ' + pytesseract.image_to_string(Image.open(img_path), timeout=10))
    return result

def img2txt_gpu(image_id):
    img_path = os.path.join('../input/shopee-product-matching/train_images/', image_id)
    reader = easyocr.Reader(['en'])
    #im = PIL.Image.open(img_path) #provide your image path
    result = reader.readtext(img_path, detail=0)
    print(result)
    #print(' OCR result : ' + pytesseract.image_to_string(Image.open(img_path), timeout=10))
    return result

In [ ]:
# it takes lots of time so I uploaeded the result of it 

# Doing OCR using GPU
# save the images text to dict
reader = easyocr.Reader(['en', 'en'])
images_text = {}
# for idx,files in enumerate(files_list):
#     print(idx)
#     img_text = reader.readtext(img_path + '/' +  files)
#     final_text = ""
#     for _, text, __ in img_text:
#         final_text += " "
#         final_text += text
#     images_text[files] = final_text
# train["img_txt"] = train["image"].apply(lambda x: img2txt_gpu(x))

train=pd.read_csv('../input/shop-train/train_final.csv')
train[['txt_img']]

## TEXT EXTRACT FROM PICTURES
### Clean the txt_img and merge it with refined title 

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
train["clean_txt_img"] = train["txt_img"].apply(lambda x: preprocess_text(x, flg_stemm=False, flg_lemm=True, ))

clean_txt_img=train["clean_txt_img"]
fdist_clean_txt_img=FreqDist()

for ele in clean_txt_img:
    ele_tok=word_tokenize(ele)
    for tok_word in ele_tok:
        fdist_clean_txt_img[tok_word]+=1

train["clean_txt_img_98"] = train["clean_txt_img"].apply(lambda x: preprocess2_text(x,0.98))



In [ ]:
train['title_N_text_img']=''

def listToString(s): 
    str1 = " "
    return (str1.join(s))
        
for idx in range(len(train)):
    if idx%10000 == 0:
        print(idx)
    tok_txt=train['clean_txt_img_98'].loc[idx].split()
    #print(tok_txt)
    tok_txt=[word for word in tok_txt if word in train['clean_txt_img_98'].loc[idx]]
    train['title_N_text_img'].loc[idx]=train['clean_title_remove_98'].loc[idx]+' '+listToString(tok_txt)


In [ ]:
train[['clean_title_remove_98','txt_img','title_N_text_img']]

In [ ]:

train

In [ ]:
import numpy as np, pandas as pd, gc
import cv2, matplotlib.pyplot as plt
import cudf, cuml, cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
print('RAPIDS',cuml.__version__)
print('TF',tf.__version__)
print('Computing text embeddings...')


In [ ]:
train=train.drop(['Unnamed: 0'],axis=1)


## Prediction results and Interpretation
### TITLE

In [ ]:
train[['title','clean_title_remove_98','title_N_text_img']]

In [ ]:

train_gf=cudf.DataFrame(train)

# model = TfidfVectorizer(stop_words='english', binary=True, max_features=25_000)
# text_embeddings = model.fit_transform(train_gf.clean_title_remove_98).toarray()
# print('text embeddings shape',text_embeddings.shape)


train_gf=cudf.DataFrame(train)

In [ ]:
def TextPredictionModel_cosSimilarity(col_name):
    
    model = TfidfVectorizer(stop_words='english', binary=True, max_features=25_000)
    text_embeddings = model.fit_transform(train_gf[col_name]).toarray()
    print('text embeddings shape',text_embeddings.shape)

    preds = []
    CHUNK = 1024*4

    print('Finding similar titles...')
    CTS = len(train)//CHUNK
    if len(train)%CHUNK!=0: CTS += 1
    for j in range( CTS ):

        a = j*CHUNK
        b = (j+1)*CHUNK
        b = min(b,len(train))
        print('chunk',a,'to',b)

        # COSINE SIMILARITY DISTANCE
        cts = cupy.matmul( text_embeddings, text_embeddings[a:b].T).T

        for k in range(b-a):
            IDX = cupy.where(cts[k,]>0.7)[0]
            o = train.iloc[cupy.asnumpy(IDX)].posting_id.values
            preds.append(o)

    del model, text_embeddings
    _ = gc.collect()
    return preds


In [ ]:
#train[['title','clean_title','clean_title_remove_98','title_N_text_img']]

pred_title=TextPredictionModel_cosSimilarity('title')
pred_98_refined_title=TextPredictionModel_cosSimilarity('clean_title_remove_98')
pred_title_N_text_img=TextPredictionModel_cosSimilarity('title_N_text_img')


In [ ]:
preds = pred_title
# pred_clean_title
# pred_98_refined_title
# pred_title_N_text_img

train['preds'] = preds
def getMetric(col):
    def f1score(row):
        n = len( np.intersect1d(row.answer,row[col]) )
        return 2*n / (len(row.answer)+len(row[col]))
    return f1score
def combine_for_sub(row):
    x = np.concatenate([row.preds])
    return ' '.join( np.unique(x) )

def combine_for_cv(row):
    x = np.concatenate([row.preds])
    return np.unique(x)
tmp = train.groupby('label_group').posting_id.agg('unique').to_dict()
train['answer'] = train.label_group.map(tmp)
train['oof'] = train.apply(combine_for_cv,axis=1)
train['f1'] = train.apply(getMetric('oof'),axis=1)
print('CV Score =', train.f1.mean() )

In [ ]:
preds = pred_title_N_text_img

train['preds'] = preds
def getMetric(col):
    def f1score(row):
        n = len( np.intersect1d(row.answer,row[col]) )
        return 2*n / (len(row.answer)+len(row[col]))
    return f1score
def combine_for_sub(row):
    x = np.concatenate([row.preds])
    return ' '.join( np.unique(x) )

def combine_for_cv(row):
    x = np.concatenate([row.preds])
    return np.unique(x)
tmp = train.groupby('label_group').posting_id.agg('unique').to_dict()
train['answer'] = train.label_group.map(tmp)
train['oof'] = train.apply(combine_for_cv,axis=1)
train['f1'] = train.apply(getMetric('oof'),axis=1)
print('CV Score =', train.f1.mean() )

In [ ]:
preds = pred_98_refined_title
# pred_title_N_text_img

train['preds'] = preds
def getMetric(col):
    def f1score(row):
        n = len( np.intersect1d(row.answer,row[col]) )
        return 2*n / (len(row.answer)+len(row[col]))
    return f1score
def combine_for_sub(row):
    x = np.concatenate([row.preds])
    return ' '.join( np.unique(x) )

def combine_for_cv(row):
    x = np.concatenate([row.preds])
    return np.unique(x)
tmp = train.groupby('label_group').posting_id.agg('unique').to_dict()
train['answer'] = train.label_group.map(tmp)
train['oof'] = train.apply(combine_for_cv,axis=1)
train['f1'] = train.apply(getMetric('oof'),axis=1)
print('CV Score =', train.f1.mean() )

In [ ]:
# !pip install ../input/shopee-external-models/Keras_Applications-1.0.8-py3-none-any.whl
# !pip install ../input/shopee-external-models/efficientnet-1.1.0-py3-none-any.whl
# import numpy as np
# import pandas as pd
# import gc
# import matplotlib.pyplot as plt
# from sklearn.neighbors import NearestNeighbors
# from sklearn.feature_extraction.text import TfidfVectorizer
# import tensorflow as tf
# import efficientnet.tfkeras as efn
# from tqdm.notebook import tqdm
# import math
# GET_CV=False
# def get_text_embeddings(df, max_features = 15500):
#     model = TfidfVectorizer(stop_words = 'english', binary = True, max_features = max_features)
#     text_embeddings = model.fit_transform(df['clean_title_remove_98'])
#     print(f'Our title text embedding shape is {text_embeddings.shape}')
#     del model
#     return text_embeddings

# # Function to get 50 nearest neighbors of each image and text and apply thresholds find in the training phase that optimize f1 cv score
# def get_neighbors(df, text_embeddings, KNN = 50):
#     # Get distances and indices from image and text embeddings
#     neighbors_model = NearestNeighbors(n_neighbors = KNN, metric = 'cosine').fit(text_embeddings)
#     text_distances, text_indices = neighbors_model.kneighbors(text_embeddings)
  
#     # Iterate through different thresholds to maximize cv, run this in interactive mode, then replace else clause with a solid threshold
#     if GET_CV:
#         predictions = []
#         for k in range(df.shape[0]):
#             # This are the original thresholds that gives 0.8035 cv (optimize with a for loop)
#             idx_text = np.where(text_distances[k,] < 0.30)[0]
#             ids_text = text_indices[k,idx_text]
#             # Get the union of boths ids
#             ids = list(ids_text)
#             posting_ids = ' '.join(df['posting_id'].iloc[ids].values)
#             predictions.append(posting_ids)
    
#     else:
#         predictions = []
#         for k in range(df.shape[0]):
#             idx_text = np.where(text_distances[k,] < 0.30)[0]
#             ids_text = text_indices[k,idx_text]
#             # Get the union of boths ids
#             ids = list(ids_text)
#             posting_ids = ' '.join(df['posting_id'].iloc[ids].values)
#             predictions.append(posting_ids)
        
#     del neighbors_model, text_distances, text_indices
#     gc.collect()
#     return df, predictions

# if GET_CV:
#     text_embeddings = get_text_embeddings(train, max_features = 15500)
# else:
#     text_embeddings = get_text_embeddings(train, max_features = 21500)

# # Get neighbors
# df, predictions = get_neighbors(train, text_embeddings, KNN = 50)
# df['matches'] = predictions
# train['matches_conv']=train['matches'].apply(lambda x: x.split())

In [ ]:
import numpy as np, pandas as pd, gc
import cv2, matplotlib.pyplot as plt
import cudf, cuml, cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
print('RAPIDS',cuml.__version__)
print('TF',tf.__version__)

In [ ]:
from skimage.filters import sobel
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, df, img_size=256, batch_size=32, path='',g_kind=0): 
        self.df = df
        self.img_size = img_size
        self.batch_size = batch_size
        self.path = path
        self.indexes = np.arange( len(self.df) )
        self.g_kind = g_kind
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        ct = len(self.df) // self.batch_size
        ct += int(( (len(self.df)) % self.batch_size)!=0)
        return ct

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X = self.__data_generation(indexes)
        return X
            
    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' 
        X = np.zeros((len(indexes),self.img_size,self.img_size,3),dtype='float32')
        df = self.df.iloc[indexes]
        for i,(index,row) in enumerate(df.iterrows()):
            img = cv2.imread(self.path+row.image)
            if self.g_kind==1:
                img = sobel(img[:,:])
            X[i,] = cv2.resize(img,(self.img_size,self.img_size)) #/128.0 - 1.0
        return X

In [ ]:
BASE = '../input/shopee-product-matching/train_images/'

WGT = '../input/effnetb0/efficientnetb0_notop.h5'
model = EfficientNetB0(weights=WGT,include_top=False, pooling='avg', input_shape=None)

embeds = []
CHUNK = 1024*4

print('Computing image embeddings...')
CTS = len(train)//CHUNK
if len(train)%CHUNK!=0: CTS += 1
for i,j in enumerate( range( CTS ) ):
    
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b,len(train))
    print('chunk',a,'to',b)
    
    train_gen = DataGenerator(train.iloc[a:b], batch_size=32, path=BASE)
    image_embeddings = model.predict(train_gen,verbose=1,use_multiprocessing=True, workers=4)
    embeds.append(image_embeddings)

    #if i>=1: break
    
del model
_ = gc.collect()
image_embeddings = np.concatenate(embeds)
print('image embeddings shape',image_embeddings.shape)

In [ ]:
train_gf=cudf.DataFrame(train)
KNN = 50
model = NearestNeighbors(n_neighbors=KNN)
model.fit(image_embeddings)

In [ ]:
preds_img = []
CHUNK = 1024*4

print('Finding similar images...')
CTS = len(image_embeddings)//CHUNK
if len(image_embeddings)%CHUNK!=0: CTS += 1
for j in range( CTS ):
    
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b,len(image_embeddings))
    print('chunk',a,'to',b)
    distances, indices = model.kneighbors(image_embeddings[a:b,])
    
    for k in range(b-a):
        IDX = np.where(distances[k,]<6.0)[0]
        #IDX = np.where(distances[k,]<3.9)[0]
        IDS = indices[k,IDX]
        o = train.iloc[IDS].posting_id.values
        preds_img.append(o)
        
del model, distances, indices, image_embeddings, embeds
_ = gc.collect()

In [ ]:
train['preds_img'] = preds_img


def getMetric(col):
    def f1score(row):
        n = len( np.intersect1d(row.answer,row[col]) )
        return 2*n / (len(row.answer)+len(row[col]))
    return f1score
def combine_for_sub(row):
    x = np.concatenate([row.preds_img])
    return ' '.join( np.unique(x) )

def combine_for_cv(row):
    x = np.concatenate([row.preds_img])
    return np.unique(x)
tmp = train.groupby('label_group').posting_id.agg('unique').to_dict()
train['answer'] = train.label_group.map(tmp)
train['oof'] = train.apply(combine_for_cv,axis=1)
train['f1'] = train.apply(getMetric('oof'),axis=1)
print('CV Score =', train.f1.mean() )


In [ ]:
tmp = train.groupby('image_phash').posting_id.agg('unique').to_dict()
train['preds_img_hash'] = train.image_phash.map(tmp)


In [ ]:



def getMetric(col):
    def f1score(row):
        n = len( np.intersect1d(row.answer,row[col]) )
        return 2*n / (len(row.answer)+len(row[col]))
    return f1score
def combine_for_sub(row):
    x = np.concatenate([row.preds_img_hash])
    return ' '.join( np.unique(x) )

def combine_for_cv(row):
    x = np.concatenate([row.preds_img_hash])
    return np.unique(x)
tmp = train.groupby('label_group').posting_id.agg('unique').to_dict()
train['answer'] = train.label_group.map(tmp)
train['oof'] = train.apply(combine_for_cv,axis=1)
train['f1'] = train.apply(getMetric('oof'),axis=1)
print('CV Score =', train.f1.mean() )

In [ ]:
train[['answer','preds','preds_img','preds_img_hash']]

# Final result 

In [ ]:
def getMetric(col):
    def f1score(row):
        n = len( np.intersect1d(row.answer,row[col]) )
        return 2*n / (len(row.answer)+len(row[col]))
    return f1score
def combine_for_sub(row):
    x = np.concatenate([row.preds,row.preds_img, row.preds_img_hash])
    return ' '.join( np.unique(x) )

def combine_for_cv(row):
    x = np.concatenate([row.preds,row.preds_img, row.preds_img_hash])
    return np.unique(x)

tmp = train.groupby('label_group').posting_id.agg('unique').to_dict()
train['answer'] = train.label_group.map(tmp)
train['oof'] = train.apply(combine_for_cv,axis=1)
train['f1'] = train.apply(getMetric('oof'),axis=1)
print('CV Score =', train.f1.mean() )



In [ ]:
#train